In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# I want to load a csv placed in ../logs/log_ear_mar.csv
# with this header timestamp;path_img;ear;mar

# And load into a dataframe using pandas


2023-09-17 15:26:18.754677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
file_path = "../logs/log_all.csv"

df = pd.read_csv(file_path, sep=";")

In [3]:
df.rename(columns={df.columns[0]: "timestamp", df.columns[1]: "path_img",df.columns[2]: "ear", df.columns[3]: "mar", df.columns[4]: "landmarks"}, inplace=True)

In [4]:
df['category'] = df['path_img'].apply(lambda x: x.split('/')[2])
df

# cast ear and mar to float
df['ear'] = df['ear'].astype(float)
df['mar'] = df['mar'].astype(float)

In [40]:
num_landmarks = 2
X_train, X_temp, y_train, y_temp = train_test_split(df[["ear","mar"]], df["category"], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [27]:
# Define the neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(num_landmarks,), name='input_layer'),  # Adjust num_landmarks to match your data
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [41]:
y_train=y_train.apply(lambda x : 1 if x == "fatiga" else 0)
y_val = y_val.apply(lambda x : 1 if x == "fatiga" else 0)
y_test = y_test.apply(lambda x : 1 if x == "fatiga" else 0)
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [33]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_val,y_val))

Epoch 1/20
9/9 [==============================] - 0s 8ms/step - loss: 0.4834 - accuracy: 0.7376 - val_loss: 0.4758 - val_accuracy: 0.8000
Epoch 2/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4798 - accuracy: 0.7447 - val_loss: 0.4738 - val_accuracy: 0.8000
Epoch 3/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4774 - accuracy: 0.7411 - val_loss: 0.4754 - val_accuracy: 0.7833
Epoch 4/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4757 - accuracy: 0.7376 - val_loss: 0.4774 - val_accuracy: 0.7833
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4717 - accuracy: 0.7447 - val_loss: 0.4758 - val_accuracy: 0.7833
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4685 - accuracy: 0.7482 - val_loss: 0.4709 - val_accuracy: 0.8167
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4662 - accuracy: 0.7482 - val_loss: 0.4725 - val_accuracy: 0.8167
Epoch 8/20
9/9 [==================

In [46]:
X_test[0]

array([0.2600315 , 0.70637419])

In [42]:


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Save the model for future use
model.save('fatigue_detection_model.h5')


2/2 [==============================] - 0s 4ms/step - loss: 0.3941 - accuracy: 0.8689
Test accuracy: 86.89%


/Users/rguarise/.pyenv/versions/3.11.0/envs/PFE/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [53]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('../models/fatigue_detection_model.h5')


1/1 [==============================] - 0s 52ms/step


array([[0.944017]], dtype=float32)

In [55]:
model.predict(X_test[1].reshape(1,2))

1/1 [==============================] - 0s 28ms/step


array([[0.13814627]], dtype=float32)